In [ ]:
!pip install llmcompressor

In [ ]:
from llmcompressor.modifiers.awq import AWQModifier
from llmcompressor import oneshot

In [ ]:
from llmcompressor.utils import dispatch_for_generation
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import torch

In [ ]:
MODEL_ID = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16).cuda()
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
DATASET_ID = "HuggingFaceH4/ultrachat_200k"
DATASET_SPLIT = "train_sft"
NUM_CALIBRATION_SAMPLES = 64
MAX_SEQUENCE_LENGTH = 256

In [ ]:
from datasets import load_dataset
ds = load_dataset(DATASET_ID, split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]").shuffle(seed=42)

README.md: 0.00B [00:00, ?B/s]

data/train_sft-00000-of-00003-a3ecf92756(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/train_sft-00001-of-00003-0a1804bcb6(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/train_sft-00002-of-00003-ee46ed25cf(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/test_sft-00000-of-00001-f7dfac4afe5(…):   0%|          | 0.00/81.2M [00:00<?, ?B/s]

...

data/test_gen-00000-of-00001-3d4cd830914(…):   0%|          | 0.00/80.4M [00:00<?, ?B/s]

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

In [ ]:
def preprocess(example):
    # Manually construct the conversation string from the list of messages.
    # The format can be adapted, but a simple role: content structure is common.
    chat_string = ""
    for message in example["messages"]:
        # Assuming each message has 'role' and 'content' keys
        chat_string += f"{message['role']}: {message['content']}\n"
    return {"text": chat_string.strip()}

ds = ds.map(preprocess)

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [ ]:
def tokenize(sample):
    return tokenizer(
        sample["text"],
        padding=False,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        add_special_tokens=False,
    )

ds = ds.map(tokenize)
ds = ds.remove_columns(["text", "messages"])

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [ ]:
recipe = [
    AWQModifier(ignore=["lm_head"], scheme="W4A16_ASYM", targets=["Linear"])
]

In [ ]:
oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

2025-11-07T09:47:07.971953+0000 | reset | INFO - Compression lifecycle reset
2025-11-07T09:47:07.976398+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-11-07T09:47:08.218643+0000 | on_initialize | INFO - No AWQModifier.mappings provided, inferring from model...
2025-11-07T09:47:08.219902+0000 | get_layer_mappings_from_architecture | INFO - Architecture OPTForCausalLM not found in mappings. Using default mappings: [AWQMapping(smooth_layer='re:.*input_layernorm$', balance_layers=['re:.*q_proj$', 're:.*k_proj$', 're:.*v_proj$']), AWQMapping(smooth_layer='re:.*v_proj$', balance_layers=['re:.*o_proj$']), AWQMapping(smooth_layer='re:.*post_attention_layernorm$', balance_layers=['re:.*gate_proj$', 're:.*up_proj$']), AWQMapping(smooth_layer='re:.*up_proj$', balance_layers=['re:.*down_proj$'])]


0it [00:00, ?it/s]
Resolving mapping 2/4 (0 skipped): : 24it [00:00, 1005.84it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

2025-11-07T09:47:08.264142+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-11-07T09:47:08.265240+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `AWQModifier`



(1/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 72.95it/s]
Smoothing: 0it [00:00, ?it/s]
(2/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 774.32it/s]
Smoothing: 0it [00:00, ?it/s]
(3/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 754.13it/s]
Smoothing: 0it [00:00, ?it/s]
(4/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 772.48it/s]
Smoothing: 0it [00:00, ?it/s]
(5/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 629.54it/s]
Smoothing: 0it [00:00, ?it/s]
(6/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 789.79it/s]
Smoothing: 0it [00:00, ?it/s]
(7/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 815.08it/s]
Smoothing: 0it [00:00, ?it/s]
(8/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 805.36it/s]
Smoothing: 0it [00:00, ?it/s]
(9/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 817.74it/s]
Smoothing: 0it [00:00, ?it/s]
(10/25): Calibrating: 100%|██████████| 64/64 [00:00<00:00, 731.88it/s]
Smoothing: 0it [00:00, ?it/s]

2025-11-07T09:47:16.429527+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2025-11-07T09:47:16.470091+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=409

In [ ]:
dispatch_for_generation(model)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=409

In [ ]:
input_ids = tokenizer("Hello my name is", return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0]))

</s>Hello my name is John.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names.
I'm a man of many names
